In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score,classification_report,confusion_matrix

In [2]:
data=pd.read_csv('spam.csv',encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [4]:
data=data.drop(columns=[ 'Unnamed: 2' , 'Unnamed: 3' ,'Unnamed: 4'])
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
data.columns=['label','text']
data

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   object
 1   text    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
data.isnull().dropna()

,label,text
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
5567,False,False
5568,False,False
5569,False,False
5570,False,False


In [8]:
y=data['label']
x=data['text']

In [9]:
le=LabelEncoder()
y_encoded=le.fit_transform(y)
y_encoded

array([0, 0, 1, ..., 0, 0, 0])

In [10]:
vectorizer=TfidfVectorizer(
    stop_words='english',
    max_features=3000
)
x_vectorized=vectorizer.fit_transform(x)
x_vectorized

<5572x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 37096 stored elements in Compressed Sparse Row format>

In [11]:
x_train,x_test,y_train,y_test=train_test_split(
    x_vectorized,y_encoded,test_size=0.2,random_state=42
)

In [12]:
preprocess_pipline= Pipeline([
    ("scaler",StandardScaler(with_mean=False))
])

In [13]:
x_train_preperd=preprocess_pipline.fit_transform(x_train)
x_test_preperd=preprocess_pipline.transform(x_test)

In [14]:
model=LogisticRegression()
model.fit(x_train_preperd,y_train)

LogisticRegression()

In [15]:
y_pred=model.predict(x_test_preperd)

In [16]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       965
           1       0.96      0.88      0.92       150

    accuracy                           0.98      1115
   macro avg       0.97      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [17]:
confusion_matrix(y_test,y_pred)

array([[960,   5],
       [ 18, 132]], dtype=int64)

In [18]:
y_prob=model.predict_proba(x_test_preperd)[:,1]
y_pred_new=(y_prob>=0.25).astype(int)

In [19]:
print(classification_report(y_test,y_pred_new))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.96      0.91      0.93       150

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [20]:
confusion_matrix(y_test,y_pred_new)

array([[959,   6],
       [ 14, 136]], dtype=int64)

In [21]:
new_data=x_vectorized[2]
sample=preprocess_pipline.transform(new_data)
y_new=model.predict(sample)
print(y_new)

[1]


In [22]:
if y_new==0:
    print("ham")
else:
    print("spam")

spam
